In [11]:
from azureml.core.dataset import Dataset
import azureml.core

In [12]:
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.51.0


In [ ]:
from azure.ai.ml import command, Input, MLClient, UserIdentityConfiguration, ManagedIdentityConfiguration
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential

# Set your subscription, resource group and workspace name:
subscription_id = '25130c3f-778b-4637-bfb8-3b1b885b45e7'
resource_group = 'rg-silo-dev-003'
workspace = 'aml-silo-dev-003'

# connect to the AzureML workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# ==============================================================
# Set the URI path for the data. Supported paths include:
# local: `./<path>
# Blob: wasbs://<container_name>@<account_name>.blob.core.windows.net/<path>
# ADLS: abfss://<file_system>@<account_name>.dfs.core.windows.net/<path>
# Datastore: azureml://datastores/<data_store_name>/paths/<path>
# Data Asset: azureml:<my_data>:<version>
# We set the path to a file on a public blob container
# ==============================================================
path = "https://cytotoxstorageaccount.blob.core.windows.net/compounds-images/reference_dataset_compounds_v2/"

# ==============================================================
# What type of data does the path point to? Options include:
# data_type = AssetTypes.URI_FILE # a specific file
# data_type = AssetTypes.URI_FOLDER # a folder
# data_type = AssetTypes.MLTABLE # an mltable
# The path we set above is a specific file
# ==============================================================
data_type = AssetTypes.URI_FILE

# ==============================================================
# Set the mode. The popular modes include:
# mode = InputOutputModes.RO_MOUNT # Read-only mount on the compute target
# mode = InputOutputModes.DOWNLOAD # Download the data to the compute target
# ==============================================================
mode = InputOutputModes.RO_MOUNT

# ==============================================================
# You can set the identity you want to use in a job to access the data. Options include:
# identity = UserIdentityConfiguration() # Use the user's identity
# identity = ManagedIdentityConfiguration() # Use the compute target managed identity
# ==============================================================
# This example accesses public data, so we don't need an identity.
# You also set identity to None if you use a credential-based datastore
identity = None

# Set the input for the job:
inputs = {
    "input_data": Input(type=data_type, path=path, mode=mode)
}

# This command job uses the head Linux command to print the first 10 lines of the file
job = command(
    command="head ${{inputs.input_data}}",
    inputs=inputs,
    environment="azureml://registries/azureml/environments/sklearn-1.1/versions/4",
    compute="cpu-cluster",
    identity=identity,
)

# Submit the command
ml_client.jobs.create_or_update(job)

In [17]:

from azureml.core import Workspace, Dataset, Datastore

subscription_id = '25130c3f-778b-4637-bfb8-3b1b885b45e7'
resource_group = 'rg-silo-dev-003'
workspace_name = 'aml-silo-dev-003'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "workspaceblobcompound")
dataset = Dataset.File.from_files(path=(datastore, 'reference_dataset_compounds_v2'))
dataset.download(target_path='.', overwrite=True)

NotImplementedError: _path_to_get_files_block is no longer supported. Deprecated, downgrade to a previous version of azureml-dataprep.

In [14]:

from azureml.core import Workspace, Dataset, Datastore

subscription_id = '25130c3f-778b-4637-bfb8-3b1b885b45e7'
resource_group = 'rg-silo-dev-003'
workspace_name = 'aml-silo-dev-003'

ws = Workspace(subscription_id, resource_group, workspace_name)
print(ws.name, ws.location, ws.resource_group, sep='\t')


aml-silo-dev-003	westeurope	rg-silo-dev-003


In [21]:
# dataset_name = "workspaceblobcompound\\paths\\reference_dataset_compounds_v2"
dataset = Dataset.get_by_name(ws, name='compound-datasetv2', version=1) #
# dataset = Dataset.File.from_files(path=(datastore, 'reference_dataset_compounds_v2'))
mounted_path = dataset.mount()
mounted_path.start()

UserErrorException: UserErrorException:
	Message: Failed to download mltable yaml with error: 
Error Code: UserError
Error Message: 
Error Code: ScriptExecution.StreamAccess.NotFound
Native Error: error in streaming from input data sources
	StreamError(NotFound)
=> stream not found
	NotFound
Error Message: The requested stream was not found. Please make sure the request uri is correct.| session_id=69f11902-871e-483f-a013-7c4b52695de1; Not able to find MLTable file| session_id=69f11902-871e-483f-a013-7c4b52695de1
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Failed to download mltable yaml with error: \nError Code: UserError\nError Message: \nError Code: ScriptExecution.StreamAccess.NotFound\nNative Error: error in streaming from input data sources\n\tStreamError(NotFound)\n=> stream not found\n\tNotFound\nError Message: The requested stream was not found. Please make sure the request uri is correct.| session_id=69f11902-871e-483f

In [22]:
from azureml.core import Workspace, Datastore, Dataset

# 从配置加载工作区
ws = Workspace.from_config()

# 获取或创建数据存储
datastore = Datastore.get(ws, datastore_name='workspaceblobcompound')

# 创建并注册数据集
dataset = Dataset.File.from_files(path=(datastore, 'reference_dataset_compounds_v2/'))
dataset = dataset.register(workspace=ws,
                           name='CytoDataset',
                           description='Sample dataset for compounds',
                           create_new_version=True)


NotImplementedError: _path_to_get_files_block is no longer supported. Deprecated, downgrade to a previous version of azureml-dataprep.

In [27]:

import mltable
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("compound-datasetv2", version="1")

path = {
  'folder': data_asset.path
}

tbl = mltable.from_delimited_files(paths=[path])


ModuleNotFoundError: No module named 'azureml.dataprep.api.mltable._validation_and_error_handler'

In [24]:
from azureml.dataprep import auto_read_file


In [20]:
import azureml.dataprep as adp
print(adp.__version__)


5.1.6


In [8]:
# 'https://cytotoxstorageaccount.blob.core.windows.net/azureml-blobstore-82432d5c-ddfd-44db-a5f4-9ad275693506'
web_path = 'azureml://subscriptions/25130c3f-778b-4637-bfb8-3b1b885b45e7/resourcegroups/rg-silo-dev-003/workspaces/aml-silo-dev-003/datastores/workspaceblobstore/paths/230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]'
#'azureml://subscriptions/25130c3f-778b-4637-bfb8-3b1b885b45e7/resourcegroups/rg-silo-dev-003/workspaces/aml-silo-dev-003/datastores/workspaceblobcompound/paths/reference_dataset_imaging_TIFFs_processed/'
# dataset = Dataset.Tabular.from_delimited_files(path=web_path)

In [10]:
data_asset = 'azureml:cytox-sampledata:1'

In [9]:
from azureml.fsspec import AzureMachineLearningFileSystem

# instantiate file system using following URI
fs = AzureMachineLearningFileSystem(web_path)

# fs.ls()

In [10]:
fs.ls()

['230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-1-001001001.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-1-001001002.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-1-001001003.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-1-001001004.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-1-001001005.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-2-001001001.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-2-001001002.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-2-001001003.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-2-001001004.tif',
 '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001001-2-001001005.tif',
 '230810GA24B0/2023-

In [11]:
selected_filelist = ['230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001003-1-001001005.tif',
                     '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/001002-2-001001004.tif',
                     '230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/005004-2-001001005.tif']

In [14]:
fs.open(selected_filelist[0])

In [17]:
from PIL import Image
Image.open(fs.open(selected_filelist[0]))
#mpimg.imread(fs.open(selected_filelist[0]))

SyntaxError: not a PNG file (<string>)

In [ ]:
fsimage_list = []
for address in selected_filelist:
    fsimage_list.append(fs.glob(address))

In [5]:
# fs.download(rpath='data/fsspec/crime-spring.csv', lpath='data/download_files/, recursive=False)
# 230810GA24B0/2023-08-11T104541+0300[4761]/2023-08-11T104541+0300[4761]/
# you need to specify recursive as True to download a folder
fs.download(rpath='230810GA24B0', lpath='tempdata/', recursive=True)

In [ ]:
import tempfile
# mounted_path = tempfile.mkdtemp()
mounted_path = './tempdata'
# mount dataset onto the mounted_path of a Linux-based compute
mount_context = dataset.mount(mounted_path)

mount_context.start()

import os
print(os.listdir(mounted_path))
print (mounted_path)